In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import json

In [2]:
DRIVER_PATH = "../driver/chromedriver"

## Scrape college links

In [ ]:
college_name_elements = {
    "colleges": []
}

index = 0

# Scroll the college search page until no more colleges are generated
try:
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get("https://waf.collegedata.com/college-search")

    time.sleep(5)

    body = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)

    while True:
        
        time.sleep(3)
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(3)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(3)

        print(new_height)

        # Scrape all college links, city, and state abbreviation if reach the bottom of the page
        if new_height == last_height:
            college_cards = driver.find_elements(by=By.CLASS_NAME, value="CollegeCard_headerContainer__rDtct")
            time.sleep(60)
            for college_card in college_cards:

                college_links_by_name = college_card.find_element(by=By.CLASS_NAME, value="Link_link__a-VS4")
                college_name = college_links_by_name.find_element(by=By.TAG_NAME, value="h3").text
                link = college_links_by_name.get_attribute("href")
                location = college_card.find_element(by=By.CLASS_NAME, value="CollegeCard_infoText__1QGg8").text
                college_name_elements["colleges"].append({
                    "college_name": college_name,
                    "link": link,
                    "city":  "" if location is None else location.split(", ")[0],
                    "state": "" if location is None else location.split(", ")[1],
                })
            time.sleep(120)
            break

        last_height = new_height
finally:
    driver.quit()
    with open("./college_links.json", "w") as f:
        f.write(json.dumps(college_name_elements, indent=4))
